In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
%matplotlib inline
import albumentations as A
import os
import numpy as np
import pandas as pd

import albumentations as A
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings  
warnings.filterwarnings('ignore')

__print__ = print

def print(string):
    os.system(f'echo \"{string}\"')
    __print__(string)

In [ ]:
EPOCHS=20
SAMPLE_LEN=100
IMAGE_PATH="../input/plant-pathology-2021-fgvc8/train_images"
TRAIN_PATH="../input/plant-pathology-2021-fgvc8/train.csv"
SUB_PATH="../input/plant-pathology-2021-fgvc8/sample_submission.csv"


In [ ]:
sub=pd.read_csv(SUB_PATH)
train_data=pd.read_csv(TRAIN_PATH)


In [ ]:
train_data.shape

In [ ]:
train_data.labels.value_counts()

In [ ]:
train_data.groupby("labels")

In [ ]:
a=train_data['labels'].unique()
b=train_data['labels'].value_counts()

# DATASET

In [ ]:
DIR_INPUT = '../input/plant-pathology-2021-fgvc8'

SEED = 42
N_FOLDS = 5
N_EPOCHS = 10
BATCH_SIZE = 64
SIZE = 224

In [ ]:
train_data.head()

In [ ]:
encoded=dict(zip(list(train_data['labels'].value_counts().keys()),range(12)))
train_data=train_data.replace({'labels':encoded})

In [ ]:
encoded

In [ ]:
train_data.head()

In [ ]:
train,valid=train_test_split(train_data,stratify=train_data.labels,test_size=0.30)
#valid,_=train_test_split(valid,stratify=valid.labels,test_size=.95)


# Percentage of each labels in the validation set

In [ ]:
((valid.labels.value_counts())/valid.shape[0])*100

# Percentage of each labels in the train set

In [ ]:
((train.labels.value_counts())/train.shape[0])*100

In [ ]:
train.shape

In [ ]:
train_data.loc[5,'labels']

# Dataset

In [ ]:
class PlantDataset(Dataset):
    def __init__(self,df,transforms=None):
        self.image_id=df.image.values
        self.labels=df.labels.values
        self.transforms=transforms
        
    def __len__(self):
        return len(self.image_id)
    
    def __getitem__(self,idx):
        path=DIR_INPUT+'/train_images'+'/'+self.image_id[idx]
        image=cv2.imread(path,cv2.IMREAD_COLOR)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        labels=self.labels[idx]
        
        if self.transforms:
            image=self.transforms(image=image)['image']
            
        return image,labels

In [ ]:
class PlantTestDataset(Dataset):
    def __init__(self,df,transforms=None):
        self.image_id=df.image.values
        self.tranforms=transforms
        
    def __len__(self):
        return len(self.image_id)
    def __getitem__(self,idx):
        path=DIR_INPUT+'/train_images'+'/'+self.image_id[idx]
        image=cv2.imread(path,cv2.IMREAD_COLOR)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if self.transforms:
            image=self.transforms(image=image)['image']
            
        return image    
        

# Model

In [ ]:
class Plantmodel(nn.Module):
    
    def __init__(self,num_classes=12):
        super().__init__()
        self.model=torchvision.models.resnet18(pretrained=True)
        in_features=self.model.fc.in_features
        self.model.fc=nn.Linear(in_features,num_classes)
    def forward(self,x):
        batch_size, C, H, W = x.shape
        x=self.model(x)
       # x=self.backbone.maxpool(x)
        
       # x=self.backbone.layer1(x)
       # x=self.backbone.layer2(x)
       # x=self.backbone.layer3(x)
       # x=self.backbone.layer4(x)
        
       # x=F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
       # x=F.dropout(x,0.25,self.training)
       # x=self.classifier(x)
        return x
        

In [ ]:
transforms_train = A.Compose([
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    # Affine
    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),

    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])


In [ ]:
train.head()

In [ ]:
train_dataset=PlantDataset(train,transforms_train)
train_loader=DataLoader(train_dataset,batch_size=64,num_workers=0,shuffle=True)

In [ ]:
valid_dataset=PlantDataset(valid,transforms_valid)
valid_loader=DataLoader(valid_dataset,batch_size=64,num_workers=0,shuffle=False)

For each epoch we go through the following steps:
  model.train()
  iterate through each batch
 
     load image and labels
     load image and labels to device
     pass image through model and get output
     calculate loss
     calculate gradient through loss.backward()
     optimizer.step()
     zero the optimizer
 


In [ ]:
device = torch.device("cuda")
model=Plantmodel()
model.to(device)

In [ ]:
x=torch.tensor([[1,2,3,5],[1,2,3,5],[1,2,3,5]])
y=torch.tensor([1,2,3])
b=torch.argmax(x,dim=1)
torch.mean((y==b).type(torch.FloatTensor))

In [ ]:

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
min_acc=0
for epoch in range(10):
    print(f"Epoch{epoch}")
    model.train()
    train_loss=0
    loop=tqdm(enumerate(train_loader),total=len(train_loader),leave=False)
    for step,batch in loop :
        optimizer.zero_grad()
        image=batch[0]
        labels=batch[1]
        
        image=image.to(device,dtype=torch.float)
        labels=labels.to(device,dtype=torch.long)
        
        output=model(image)
        pred=torch.exp(output)
        pred=torch.argmax(output,dim=1)
        acc=torch.mean(((labels==pred).type(torch.FloatTensor)))
       # print(f" Training Accuracy {acc*100}%")
        loss=criterion(output,labels)
        loop.set_postfix(loss=loss.item(),accuracy=acc)
       
        
       
        loss.backward()
        optimizer.step()
    model.eval()
    for data,labels in valid_loader:
        image,labels=data,labels
        image=image.to(device,dtype=torch.float)
        labels=labels.to(device,dtype=torch.long)
        
        output=model(image)
        pred=torch.exp(output)
        pred=torch.argmax(output,dim=1)
        acc=torch.mean(((labels==pred).type(torch.FloatTensor)))
        loss=criterion(output,labels)
        valid_loss=loss.item()*data.size(0)
        print(f" Validation Accuracy {acc*100}%")
        
        if min_acc< acc:
            min_acc=acc
            print(f"Validation accuracy increased {min_acc*100:.2f} --> {acc*100:.2f} \t Saving Yhe Model")
            torch.save(model.state_dict(),'saved_model.pth')

In [ ]:
torch.save(model.state_dict(),'Plant_2021_epoch10.pth')